In [ ]:
# This file contains tests for `CharismaMixin`
import os
import sys
import shutil
import warnings
warnings.filterwarnings('ignore')

import numpy as np

sys.path.insert(0, '../../../seismiqb')

from seismiqb import Field, array_to_segy

In [ ]:
# Defaults for run this notebook directly
# Data creation parameters
CUBE_SHAPE = (100, 100, 100)
SEED = 42

# Preparation: workspace and fake data creation

**Storage structure:**
___



**charisma_test_files** (charisma tests directory with temporary files)

&emsp;├── **test_array.npy** (Test cube data, now it is a random noise)

&emsp;├── **test_cube.sgy** (Test cube data in the sgy format)

&emsp;├── **test_cube.meta** (Meta data of the test cube)
    
&emsp;├── **test_matrix** (Saved test matrix)

&emsp;└── **test_points** (Saved test points)


In [ ]:
# (Re)create the test directory
OUTPUT_DIR = './charisma_test_files'

if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

os.makedirs(OUTPUT_DIR)

In [ ]:
%%time
# Create a fake cube
CUBE_PATH = os.path.join(OUTPUT_DIR, 'test_cube.sgy')

rng = np.random.default_rng(SEED)
data_array = rng.normal(0, 1000, CUBE_SHAPE).astype(np.float32)

with open(os.path.join(OUTPUT_DIR, 'test_array.npy'), 'wb') as outfile:
    np.save(outfile, data_array)

array_to_segy(array_like=data_array, path=CUBE_PATH, zip_segy=False, pbar='t')

In [ ]:
%%time
# Init a field
field = Field(geometry=CUBE_PATH)

# Tests

In [ ]:
%%time
# Test dump_charisma, load_charisma and is_charisma_like (matrix case)
# Create and dump a random matrix
matrix = rng.integers(low=0, high=10, size=CUBE_SHAPE[:2])

matrix_path=os.path.join(OUTPUT_DIR, 'test_matrix')

field.dump_charisma(data=matrix, path=matrix_path, format='matrix', name='saved_matrix')

# Check is_charisma_like
assert field.is_charisma_like(matrix_path), "Dumping matrix test: the dumped matrix file is not charisma-like."

# Open saved data
opened_matrix = field.load_charisma(path=matrix_path, format='matrix')

assert np.array_equal(matrix, opened_matrix), "Dumping matrix test: saved and loaded matrices are not the same."

In [ ]:
%%time
# Test dump_charisma, load_charisma and is_charisma_like (points case)
# Create random points in the field
i_lines, x_lines = np.mgrid[0:CUBE_SHAPE[0], 0:CUBE_SHAPE[1]]

points_column_shape = (i_lines.size, 1)
i_lines, x_lines = i_lines.reshape(points_column_shape), x_lines.reshape(points_column_shape)

depths = rng.integers(low=0, high=10, size=points_column_shape)

points = np.hstack([i_lines, x_lines, depths])

# Save points as charisma
points_path=os.path.join(OUTPUT_DIR, 'test_points')

field.dump_charisma(data=points, path=points_path, format='points', name='saved_points')

# Check is_charisma_like
assert field.is_charisma_like(points_path), "Dumping points test: the dumped points file is not charisma-like."

# Open saved data
opened_points = field.load_charisma(path=points_path, format='points')

assert np.array_equal(points, opened_points), "Dumping points test: saved and loaded points arrays are not the same."